<h1>
  <center>
    CLASIFICACIÓN DE VOZ A PARTIR DE GRABACIONES CON SONIDO AMBIENTAL
  </center>
</h1>
<p>
  <center>
    Víctor Alfonso Mantilla Villamizar<br>
    Código: 2151846<br>
    Escuela de Ingeniería de Sistemas<br>
    Universidad Industrial de Santander<br>
    2019
  </center>
</p>

<h2>
  1. Introducción
</h2>
<p align="justify">
  En el siguiente proyecto se trata de crear una <b>inteligencia artificial </b> <sup><a href="https://es.wikipedia.org/wiki/Inteligencia_artificial" target="_blank">[1]</a></sup>  que tomando como base un archivo de audio cuyo contenido sea principalmente conversaciones grabadas en un entorno ruidoso en cualquier escala, sea capaz de determinar en que momentos de la grabación una o varias personas están hablando.<br>
  Para este propósito, primero se crea un dataset en base a información existente en internet y luego se crean diferentes modelos de inteligencias artificiales, para determinar cual de ellos es el mejor dada la solución buscada, y por último se presentan los resultados probando dicho modelo con varios archivos de audio y determinando la exactitud del modelo escogido.
</p>

<h2>
  2. Motivación
</h2>
<p align="justify">
  En el mundo moderno donde la Web 2.0 <sup><a href="https://es.wikipedia.org/wiki/Web_2.0" target="_blank">[2]</a></sup> domina todos los campos de nuestras vidas diarias, el video y el audio, juntos o separados constituyen no sólo una fuente de entretenimiento sino también de educación a distancia, de culturización y de comunicación. Sin embargo, algunas personas pueden quedar excluídas de las ventajas mencionadas, como por ejemplo aquellas que deseen consumir algún producto que esté en otro idioma que desconoce o también si posee dificultades auditivas.<br>
  Es debido a esto que surge la motivación de ayudar en la creación de una herramienta que permita determinar los momentos de una grabación de audio en los cuales ocurre una conversación o un monólogo, para ayudar a la fácil creación de archivos de texto de apoyo para las personas que así lo requieran.
</p>

<h2>
  3. Temas Abordados
</h2>
<p align="justify">
  En este proyecto se abordan los temas:
  <ul>
    <li>Datasets (creación y manipulación de un dataset)</li>
    <li>Modelos de clasificación (Prueba de todos los modelos vistos para hallar el adecuado)</li>
  </ul>
</p>

<h2>
  4. Funcionamiento
</h2>
<h3>
    4.1 Creación del Dataset
</h3>
<p align="justify">
  El archivo a usar para crear el dataset es un archivo de audio de más de una hora de duración y un archivo de texto que contiene los tiempos en los cuales existe diálogo en este audio. este archivo está alojado en Google Drive y para poder usarlo, se ejecuta la celda:
</p>

Se importan los paquetes necesarios

In [0]:
import librosa
import keras
import numpy      as np
import tensorflow as tf

from google.colab                      import drive
from matplotlib                        import pyplot                 as plt
from tensorflow                        import keras
from keras.models                      import Sequential
from keras.layers                      import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.optimizers                  import SGD
from keras.layers.advanced_activations import LeakyReLU


Se carga el archivo base del dataset y se crea una imagen de la amplitud del audio en decibelios

In [5]:
drive.mount('/content/drive')
signal, rate = librosa.load("drive/My Drive/Datasets/audio/audio.wav", sr=48000)
signal=librosa.core.resample(signal,rate,22050)
rate=22050
hop = 256
win_siz= 1024
window = np.hanning(win_siz)
stft = librosa.stft(signal, n_fft = win_siz, hop_length = hop, window=window)
stft_magnitude, stft_phase = librosa.magphase(stft)
image = librosa.amplitude_to_db(stft_magnitude)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Aquí se muestra una parte de la imagen generada.

In [0]:
plt.figure(figsize=[100,100])
plt.imshow(image[:,0:10000])

Algunas funciones necesarias para crear las etiquetas del dataset

In [0]:
def timetomilis(time):
  parts=time.split(":")
  if(len(parts)==3):
    return (int(parts[0])*3600000)+(int(parts[1])*60000)+(int(parts[2].replace(",","")))

In [0]:
def get_labels(total,rate,filename):
  file_text = open(filename,"r")
  text=file_text.read()
  lines=text.split("\n")
  labels=np.zeros(total)
  counter=0;
  for i in range(len(lines)):
    start_stop=lines[i].split("\t")
    start_sample=int(timetomilis(start_stop[0])*rate/1000)
    stop_sample=int(timetomilis(start_stop[1])*rate/1000)
    labels[start_sample:stop_sample]=1
    counter=counter+(stop_sample-start_sample)
  percentage=100*counter/total
  print("Porcentaje de audio con diálogos: "+str(round(percentage,2)))
  return labels

Con esta función se crean las características (X) y las etiquetas (y)

In [0]:
def get_X_y(image,labels):
  a,b=np.shape(image)
  c=len(labels)
  d=int(b/25)
  e=int(c/b)
  if(c<b*e):
    labels=np.append(labels,np.zeros(b*e-c))
  y=np.reshape(labels[0:b*e],[b,e])
  y=np.mean(y,axis=1)
  img=image[:,(y==0)|(y==1)]
  y=y[(y==0)|(y==1)]
  a,b=np.shape(img)
  c=len(labels)
  d=int(b/25)
  e=int(c/b)
  X=np.reshape(img[:,0:25*d],[a,25,d],order='F')
  y=np.reshape(y[0:25*d],[25,d],order='F')
  y=np.mean(y,axis=0)
  X=X[:,:,(y==0)|(y==1)]
  y=y[(y==0)|(y==1)]
  X=np.transpose(X,[2,0,1])
  return X,y

Por último, se obtiene X y y

In [12]:
labels=get_labels(len(signal),rate,"drive/My Drive/Datasets/audio/times.txt")
X,y=get_X_y(image,labels)
print(np.shape(X))

Porcentaje de audio con diálogos: 41.5
(16320, 513, 25) (16320,)
(16320, 513, 25)


<h3>
    4.2 Creación del modelo
</h3>
<p align="justify">
  Usando el módulo keras para python, se crea un modelo de Deep Learning para el procesamiento de los datos. Este modelo se considera el mejor para el trabajo, después de haber analizado los demás modelos de Machine Learning.
</p>

In [0]:
model = Sequential()
model.add(Conv2D(16, (3,3), padding='same', input_shape=(513, 25, 1)))
model.add(LeakyReLU())
model.add(Conv2D(16, (3,3), padding='same'))
model.add(LeakyReLU())
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))
model.add(Conv2D(16, (3,3), padding='same'))
model.add(LeakyReLU())
model.add(Conv2D(16, (3,3), padding='same'))
model.add(LeakyReLU())
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64))
model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss=keras.losses.binary_crossentropy, optimizer=sgd, metrics=['accuracy'])


<h3>
    4.3 Entrenamiento
</h3>
<p align="justify">
  Se entrena el modelo con parte del dataset creado, teniendo la precaución de mezclarlos para garantizar aleatoriedad. Con la función siguiente se obtiene valores para train, test y reservados para la prueba del modelo.
</p>

In [0]:
def get_train_test_val(X,y,test_size=0.2):
  a=len(y)
  print(a)
  c=np.random.permutation(a)
  d=int(test_size*a)
  X_test=X[c[0:d],:]
  X_train=X[c[d:a],:]
  y_test=y[c[0:d]]
  y_train=y[c[d:a]]
  X_val = X_train[-1000:,:]
  y_val = y_train[-1000:]
  X_train = X_train[:-1000,:]
  y_train = y_train[:-1000]
  return X_train,y_train,X_test,y_test,X_val,y_val


Se consigue al fin las particiones de X_train y y_train necesarias para entrenar el modelo.

In [0]:
X_train,y_train,X_test,y_test,X_val,y_val=get_train_test_val(X,y)
a,b,c=np.shape(X_train)
X_t=np.reshape(X_train,[a,b,c,1])
y_t=np.copy(y_train)
a,b,c=np.shape(X_val)
X_v=np.reshape(X_val,[a,b,c,1])
y_v=np.copy(y_val)
model.fit(X_t, y_t, epochs=50, verbose=1,validation_data=(X_v, y_v))

16320
Train on 12056 samples, validate on 1000 samples
Epoch 1/50
12056/12056 [==============================] - 127s 11ms/step - loss: 0.7168 - acc: 0.5934 - val_loss: 0.6704 - val_acc: 0.6090
Epoch 2/50
12056/12056 [==============================] - 126s 10ms/step - loss: 0.6426 - acc: 0.6340 - val_loss: 0.6151 - val_acc: 0.6660
Epoch 3/50
12056/12056 [==============================] - 125s 10ms/step - loss: 0.6326 - acc: 0.6516 - val_loss: 0.6140 - val_acc: 0.6530
Epoch 4/50
12056/12056 [==============================] - 126s 10ms/step - loss: 0.6232 - acc: 0.6604 - val_loss: 0.6095 - val_acc: 0.6620
Epoch 5/50
12056/12056 [==============================] - 126s 10ms/step - loss: 0.6206 - acc: 0.6622 - val_loss: 0.6120 - val_acc: 0.6540
Epoch 6/50
12056/12056 [==============================] - 126s 10ms/step - loss: 0.6146 - acc: 0.6703 - val_loss: 0.6129 - val_acc: 0.6700
Epoch 7/50
12056/12056 [==============================] - 125s 10ms/step - loss: 0.6147 - acc: 0.6749 - val_los

<h3>
    4.3 Prueba
</h3>
<p align="justify">
Para las pruebas del modelo se usan dos fuentes de datos diferentes: datos extraídos del mismo conjunto que los datos de entrenamiento y también extraídos de una fuente secundaria y diferente. Se obtiene un resultado aceptable para los datos prpios y resultados insuficientes para los datos externos.
</p>

In [0]:
a,b,c=np.shape(X_test)
X_te=np.reshape(X_test,[a,b,c,1])
y_te=np.copy(y_test)
y_pred=model.predict(X_te)
mean=np.mean(y_pred)
y_pred=np.where(y_pred<mean,0,1)
print("Test accuracy: "+str(np.mean (y_pred[y_te==1] == y_te[y_te==1])))

In [0]:
signal2, rate2 = librosa.load("drive/My Drive/Datasets/audio/second_source/Audio_2.wav", sr=48000)
signal2=librosa.core.resample(signal2,rate2,22050)
rate2=22050
stft2 = librosa.stft(signal2, n_fft = win_siz, hop_length = hop, window=window)
stft_magnitude2, stft_phase2= librosa.magphase(stft2)
image2 = librosa.amplitude_to_db(stft_magnitude2)

In [0]:
labels2=get_labels(len(signal2),rate2,"drive/My Drive/Datasets/audio/second_source/text_2.txt")
X2,y2=get_X_y(image2,labels2)

In [0]:
a,b,c=np.shape(X2)
X_te2=np.reshape(X2,[a,b,c,1])
y_te2=np.copy(y2)
y_pred2=model.predict(X_te2)
mean=np.mean(y_pred2)
y_pred2=np.where(y_pred2<mean,0,1)
print("Test accuracy: "+str(np.mean (y_pred2[y_te2==1] == y_te2[y_te2==1])))